In [55]:
import pandas as pd

In [56]:
df = pd.read_csv("s3://social-signals-dev-data/year=2023/month=05/day=15/reddit.csv")
df.head()

,Unnamed: 0,submission_id,submission_url,submission_subreddit,submission_title,submission_score,submission_num_comments,entities,title_sentiment_prediction,title_sentiment_score,title_emotion_prediction,title_emotion_score,title_esg_categories_prediction,title_esg_categories_score,comments,comments_emotion_counter
0,0,13hl6of,https://www.news.com.au/lifestyle/real-life/ne...,news,Peru to send Dutch killer Joran van der Sloot ...,16420,570,"{""location"": [""Peru"", ""US""], ""person"": [""Joran...",neutral,0.75,anger,0.41,Community Relations,0.92,About time…. Deserves a harsh sentence since h...,"{""neutral"": 1}"
1,1,13htcrm,https://www.whio.com/news/local/ohio-catholic-...,news,Ohio Catholic priest convicted of sex traffick...,12340,958,"{""location"": [""Ohio""]}",negative,0.73,disgust,0.90,Community Relations,0.98,>\tZacharias then began to groom the boys for ...,"{""neutral"": 2}"
2,2,13ibgc1,https://www.cnn.com/2023/05/15/us/florida-teac...,news,Florida teacher says she is under investigatio...,11610,1037,"{""location"": [""Florida""]}",negative,0.71,sadness,0.33,Community Relations,0.97,So all parents signed a permission slip. Then ...,"{""fear"": 1}"
3,3,13i6ypd,https://abcnews.go.com/International/wireStory...,news,"Serbia: Guns, grenades and rocket launchers am...",4326,507,"{""location"": [""Serbia""]}",negative,0.81,anger,0.37,Natural Capital,0.44,So a country of almost 7 million people with a...,"{""fear"": 1, ""sadness"": 1}"
4,4,13hxhtr,https://www.independent.co.uk/news/world/ameri...,news,Chilling footage shows white supremacist group...,3739,1076,"{""organization"": [""Patriot Front""], ""location""...",negative,0.53,disgust,0.74,Climate Change,0.95,These people are so brave they wear clothing a...,"{""neutral"": 2, ""surprise"": 1, ""disgust"": 1}"


In [57]:
row = df.iloc[0]
row

Unnamed: 0                                                                         0
submission_id                                                                13hl6of
submission_url                     https://www.news.com.au/lifestyle/real-life/ne...
submission_subreddit                                                            news
submission_title                   Peru to send Dutch killer Joran van der Sloot ...
submission_score                                                               16420
submission_num_comments                                                          570
entities                           {"location": ["Peru", "US"], "person": ["Joran...
title_sentiment_prediction                                                   neutral
title_sentiment_score                                                           0.75
title_emotion_prediction                                                       anger
title_emotion_score                                              

In [58]:
entities = row["entities"]
entities

'{"location": ["Peru", "US"], "person": ["Joran van der Sloot", "Natalee Holloway"]}'

In [59]:
import json

res = json.loads(entities)
res

{'location': ['Peru', 'US'],
 'person': ['Joran van der Sloot', 'Natalee Holloway']}